# AWS EC2 cheatsheet

In [ ]:
import json

import boto3
import paramiko

In [ ]:
from scraper_place.config import CONFIG_AWS_EC2

In [ ]:
def pprint_json(data):
    print(json.dumps(data, indent=4))

In [ ]:
ec2_client = boto3.client(
    'ec2',
    aws_access_key_id=CONFIG_AWS_EC2['aws_access_key_id'],
    aws_secret_access_key=CONFIG_AWS_EC2['aws_secret_access_key'],
    region_name=CONFIG_AWS_EC2['region_name'],
)
ec2_client

In [ ]:
response = ec2_client.run_instances(
    BlockDeviceMappings=[
        {
            'DeviceName': '/dev/xvda',
            'Ebs': {
                'DeleteOnTermination': True,
                'VolumeSize': 8,
                'VolumeType': 'gp2',
            },
        },
    ],
    ImageId='ami-5ce55321',
    InstanceType='t2.large',
    KeyName=CONFIG_AWS_EC2['key_name'],
    MaxCount=1,
    MinCount=1,
    SecurityGroupIds=[CONFIG_AWS_EC2['security_group']],
    #DryRun=True,
)


In [ ]:
instance_id = response['Instances'][0]['InstanceId']
instance_id

In [ ]:
waiter = ec2_client.get_waiter('instance_status_ok')

In [ ]:
waiter.wait(
    InstanceIds=[instance_id]
)

In [ ]:
response = ec2_client.describe_instance_status(
    InstanceIds=[instance_id]
)

In [ ]:
response

In [ ]:
response = ec2_client.describe_instances(
    InstanceIds=[instance_id]
)

In [ ]:
ec2_ipv4 = response['Reservations'][0]['Instances'][0]['PublicIpAddress']
ec2_ipv4

In [ ]:
ssh_client = paramiko.SSHClient()

In [ ]:
ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

In [ ]:
ssh_key = paramiko.RSAKey.from_private_key_file(CONFIG_AWS_EC2['private_key'])

In [ ]:
ssh_client.connect(
    hostname=ec2_ipv4,
    username="ec2-user",
    pkey=ssh_key
)

In [ ]:
ssh_channel = ssh_client.get_transport().open_session()
ssh_channel.exec_command('wget http://apache.crihan.fr/dist/tika/tika-server-1.17.jar')
ssh_channel.recv_exit_status()

In [ ]:
ssh_channel = ssh_client.get_transport().open_session()
ssh_channel.exec_command('sudo yum -y install java')
ssh_channel.recv_exit_status()

In [ ]:
ssh_channel = ssh_client.get_transport().open_session()
ssh_channel.exec_command('java -jar tika-server-1.17.jar --host=*')

In [ ]:
ssh_client.close()

In [ ]:
response = ec2_client.terminate_instances(
    InstanceIds=[instance_id],
)

In [ ]:
response